## Preprocessing

In [1]:
import pandas as pd
import numpy as np
#
import warnings
warnings.filterwarnings("ignore")
#

In [2]:
path_to_file = "../data/Component_Faults_Data.csv"
df = pd.read_csv(path_to_file)

In [3]:
x = df.iloc[:, :48].values
y = df["class"].values.reshape(-1, 1)

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)

In [5]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

## Newer Solution for testing NN-Configuration

In [7]:
import sys
    
    
class NeuralNetworkParameterTester:

    # params for building model
    config = {
        'input_layer_dim': 48,
        'output_layer_dim': 11,
        'number_of_hidden_layers': 1,
        'number_of_units_per_layer': 10,
        'epochs': 100,
        'batch_size': 64,
        'activation_function': 'relu',
        'loss_function': 'mean_squared_error',
        'optimizer': 'sgd'}

    
    test_param_name = None  # Name of the div parameter
    test_param_val = None  # Name of the div parameter
    
    result = []  # Result of the test
    
    # schema of result
    

    # [
    #    [
    #        'test_param_val': 1,
    #        'params': [
    #            'input_layer_dim': 48,
    #            'output_layer_dim': 11,
    #            'activation_function': 'relu',
    #            'loss_function': 'mean_squared_error',
    #            'optimizer': 'sgd',
    #            'number_of_hidden_layers': 1,
    #            'number_of_units_per_layer': 10]
    #        'result': [
    #            'model': model,
    #            'acurracy': 0.923
    #        ]
    #    ],
    #    [
    #        'test_param_val': 2,
    #        'params': [
    #            'input_layer_dim': 48,
    #            'output_layer_dim': 11,
    #            'activation_function': 'relu',
    #            'loss_function': 'mean_squared_error',
    #            'optimizer': 'sgd',
    #            'number_of_hidden_layers': 2,
    #            'number_of_units_per_layer': 10]
    #        'result': [
    #            'model': model,
    #            'acurracy': 0.923
    #        ]
    #    ],
    #   [...........]
    # ]
    
    
    
    def __init__(self, test_param_name, test_param_val):
        
        # 'param_name' and 'param_val' must be of right type and not None
        # if this is the case, they will be set
        
        if test_param_name is not None and isinstance(test_param_name, str):
            self.test_param_name = test_param_name
            if test_param_val is not None and (isinstance(test_param_val, int) or isinstance(test_param_val, list)):
                self.test_param_val = test_param_val
                self.config[self.test_param_name] = self.test_param_val
            else:
                print("'test_param_val' must be of type int or list")
                sys.exit(0)
        else:
            print("'param_name' must be of type str")
            sys.exit(0)
        #self.result.append({"test_param_name": self.test_param_name})
        
    def run(self):
        
        for val in self.config[self.test_param_name]:
            
            print(val)
            
            # prepare config for individual test
            test_config = self.config  
            test_config[self.test_param_name] = val
        
            model = self.__build(test_config)
            trained_model, history = self.__train(test_config, model)
            accuracy = self.__test(trained_model)
            
            # save result
            # TODO tes param gets overritwn
            self.result.append({"test_param_val": val, 
                                'params': test_config.copy(), 
                                "result": {'model': trained_model, 
                                           'accuracy': accuracy}})


    def __build(self, test_config):

        from keras.models import Sequential
        from keras.layers import Dense

        model = Sequential()
        
        
        # Input layer
        # TODO output here
        model.add(Dense(test_config['number_of_units_per_layer'], 
                        input_dim=test_config['input_layer_dim'], 
                        activation=test_config['activation_function']))
        # Hidden layer
        for i in range(test_config['number_of_hidden_layers']):
            model.add(Dense(test_config['number_of_units_per_layer'], 
                            activation=test_config['activation_function']))
        # Output layer
        model.add(Dense(test_config['output_layer_dim'], 
                        activation="softmax"))

        model.compile(loss=test_config['loss_function'], 
                      optimizer=test_config['optimizer'], 
                      metrics=['accuracy'])

        return model

    def __train(self, test_config, model):
        # self.param_list.update({"epochs": epochs, "batch_size": batch_size})
                                    
        # xtrain and ytrain are from preprocessing
        history = model.fit(x_train, 
                            y_train, 
                            epochs=test_config['epochs'], 
                            batch_size=test_config['batch_size'])
                                    
        return model, history
    
    @staticmethod
    def __test(model):
        y_pred = model.predict(x_test)
        # Converting predictions to label
        pred = list()
        for i in range(len(y_pred)):
            pred.append(np.argmax(y_pred[i]))
        # Converting one hot encoded test label to label
        test = list()
        for i in range(len(y_test)):
            test.append(np.argmax(y_test[i]))

        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(pred, test)
        return accuracy
                                    
                                    
    # Setter methods for setting single parameters
    
    def set_number_of_hidden_layers(self, val):
        self.config["number_of_hidden_layers"] = val
        
    def set_number_of_units_per_layer(self, val):
        self.config["number_of_units_per_layer"] = val
        
    def set_activation_function(self, val):
        self.config["activation_function"] = val
        
    def set_epochs(self, val):
        self.config["epochs"] = val
        
    def set_batch_size(self, val):
        self.config["batch_size"] = val
        
    def set_loss_function(self, val):
        self.config["loss_function"] = val
        
    def set_optimizer(self, val):
        self.config["optimizer"] = val
        
    def reset_config(self):
        config = {
            'input_layer_dim': 48,
            'output_layer_dim': 11,
            'number_of_hidden_layers': 1,
            'number_of_units_per_layer': 10,
            'epochs': 100,
            'batch_size': 64,
            'activation_function': 'relu',
            'loss_function': 'mean_squared_error',
            'optimizer': 'sgd'}
        
        config[self.test_param_name] = self.test_param_val
        
        self.config = config
        
    # Plot result
    
    def plot_result(self, dim=2):
        if dim == 2:
            self.__plot_2d()

    def __plot_2d(self):

        import matplotlib.pyplot as plt

        test_param_val = [element['test_param_val'] for element in self.result]
        accuracy_result = [element['result']['accuracy'] for element in self.result]
        
        print(test_param_val)
        print(accuracy_result)

        plt.plot(test_param_val, accuracy_result)
        plt.title("Accuracy per " + "'" + self.test_param_name + "'")
        plt.ylabel("Accuracy")
        plt.xlabel(self.test_param_name)
        plt.show()
    

## Test NN with parameters

In [8]:
nnpt = NeuralNetworkParameterTester(test_param_name="number_of_hidden_layers", test_param_val=[2, 4])
nnpt.set_number_of_units_per_layer(48)
nnpt.set_epochs(100)
nnpt.run()

2


Using TensorFlow backend.









Epoch 1/100





26329/26329 [==============================] - 1s 37us/step - loss: 0.0833 - acc: 0.0910
Epoch 2/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0828 - acc: 0.1071
Epoch 3/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0822 - acc: 0.1391
Epoch 4/100
26329/26329 [==============================] - 0s 16us/step - loss: 0.0816 - acc: 0.1648
Epoch 5/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0808 - acc: 0.1744
Epoch 6/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0799 - acc: 0.1776
Epoch 7/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0788 - acc: 0.1804
Epoch 8/100
26329/26329 [==============================] - 0s 16us/step - loss: 0.0778 - acc: 0.1853
Epoch 9/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.0770 - acc: 0.1915
Epoch 10/100
26329/26329 [==============================] - 0s 18us/step - loss

26329/26329 [==============================] - 0s 15us/step - loss: 0.0602 - acc: 0.4446
Epoch 59/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0599 - acc: 0.4499
Epoch 60/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0595 - acc: 0.4553
Epoch 61/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0592 - acc: 0.4612
Epoch 62/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0589 - acc: 0.4667
Epoch 63/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0586 - acc: 0.4733
Epoch 64/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0582 - acc: 0.4787
Epoch 65/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0579 - acc: 0.4833
Epoch 66/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0576 - acc: 0.4877
Epoch 67/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.0572 - acc: 

26329/26329 [==============================] - 0s 18us/step - loss: 0.0730 - acc: 0.3042
Epoch 39/100
26329/26329 [==============================] - 0s 18us/step - loss: 0.0728 - acc: 0.3091
Epoch 40/100
26329/26329 [==============================] - 0s 19us/step - loss: 0.0726 - acc: 0.3127
Epoch 41/100
26329/26329 [==============================] - 0s 18us/step - loss: 0.0725 - acc: 0.3159
Epoch 42/100
26329/26329 [==============================] - 0s 18us/step - loss: 0.0723 - acc: 0.3177
Epoch 43/100
26329/26329 [==============================] - 0s 18us/step - loss: 0.0721 - acc: 0.3192
Epoch 44/100
26329/26329 [==============================] - 0s 18us/step - loss: 0.0719 - acc: 0.3198
Epoch 45/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.0716 - acc: 0.3186
Epoch 46/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.0714 - acc: 0.3179
Epoch 47/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.0711 - acc: 

In [9]:
# TODO sis isch falsch
nnpt.config

{'input_layer_dim': 48,
 'output_layer_dim': 11,
 'number_of_hidden_layers': 4,
 'number_of_units_per_layer': 48,
 'epochs': 100,
 'batch_size': 64,
 'activation_function': 'relu',
 'loss_function': 'mean_squared_error',
 'optimizer': 'sgd'}

In [10]:
nnpt.result

[{'test_param_val': 2,
  'params': {'input_layer_dim': 48,
   'output_layer_dim': 11,
   'number_of_hidden_layers': 2,
   'number_of_units_per_layer': 48,
   'epochs': 100,
   'batch_size': 64,
   'activation_function': 'relu',
   'loss_function': 'mean_squared_error',
   'optimizer': 'sgd'},
  'result': {'model': <keras.engine.sequential.Sequential at 0x20b96347848>,
   'accuracy': 0.6582365003417635}},
 {'test_param_val': 4,
  'params': {'input_layer_dim': 48,
   'output_layer_dim': 11,
   'number_of_hidden_layers': 4,
   'number_of_units_per_layer': 48,
   'epochs': 100,
   'batch_size': 64,
   'activation_function': 'relu',
   'loss_function': 'mean_squared_error',
   'optimizer': 'sgd'},
  'result': {'model': <keras.engine.sequential.Sequential at 0x20b9f453b48>,
   'accuracy': 0.6555023923444976}}]

In [11]:
nnpt.plot_result()

[2, 4]
[0.6582365003417635, 0.6555023923444976]


<Figure size 640x480 with 1 Axes>